In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pytube import YouTube

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import Image

from collections import deque

def bfs(img,visited,i,j):
    queue = deque([(i,j)])
    visited[i][j] = True

    while queue:
        x, y = queue.popleft()
        for nx in range(x - 1, x + 2):  
            for ny in range(y - 1, y + 2): 
                if 0 <= nx < len(img) and 0 <= ny < len(img[0]):
                    if not visited[nx][ny] and img[x,y] == img[nx,ny]:
                        visited[nx][ny] = True
                        queue.append((nx, ny))                 

def all_components(img):
    rows, cols = img.shape
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    components = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                bfs(img,visited,i,j)
                components+=1

    return components

def objCollision(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()
    frame_count = 0
    pastComponents = 0
    while True:
        ret, frame = cap.read()
        current_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        if not ret:
            print("không chạm nhau")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_frame, (5, 5), 0)
        _, binary = cv2.threshold(blurred, 147, 255, cv2.THRESH_BINARY_INV)

        currentComponents = all_components(binary)
        if pastComponents > currentComponents:
            print(f"Chạm nhau tại: {current_time_ms}, Frame: {frame_count}")
            break
        pastComponents = currentComponents
        frame_count+=1

In [3]:
import cv2
import os
import numpy as np
import pytictoc

# Đường dẫn đến video
video_path = 'videoplayback.mp4'

# Thư mục lưu ảnh
output_folder = 'frames_output'
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

# Mở video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Không thể mở video.")
    exit()

frame_count = 0  # Đếm số frame
meanTime = 00

while True:
    
    ret, frame = cap.read()
    if not ret:
        print("Video đã kết thúc.")
        break
    # h, w, _ = frame.shape
    # frame[0:h//4, 0:w] = (255, 255, 255)

    # Chuyển frame sang ảnh xám
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    _, binary = cv2.threshold(blurred, 147, 255, cv2.THRESH_BINARY_INV)
    # edges = cv2.Canny(binary, 50, 100)
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)


    t = pytictoc.TicToc()
    t.tic()
    # numsC , label = cv2.connectedComponents(binary, connectivity= 8)
    numsC = all_components(binary)
    # Tính toán gradient với Canny
    # grad_magnitude = np.abs(cv2.Laplacian(gray_frame, cv2.CV_64F).astype(np.uint8))
    meanTime += t.tocvalue()

    # Lưu frame vào thư mục
    frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.png')
    cv2.imwrite(frame_filename, binary)
    print(f"Frame {frame_count}: {numsC}")

    frame_count += 1  # Tăng bộ đếm
    if(frame_count == 56):
        break

print(f"Đã lưu {frame_count} frame vào thư mục '{output_folder}'.")
meanTime /= frame_count
print(meanTime)
cap.release()


Frame 0: 29
Frame 1: 29
Frame 2: 29
Frame 3: 29
Frame 4: 29
Frame 5: 29
Frame 6: 29
Frame 7: 29
Frame 8: 29
Frame 9: 29
Frame 10: 29
Frame 11: 29
Frame 12: 29
Frame 13: 31
Frame 14: 31
Frame 15: 31
Frame 16: 31
Frame 17: 31
Frame 18: 31
Frame 19: 31
Frame 20: 31
Frame 21: 31
Frame 22: 31
Frame 23: 31
Frame 24: 31
Frame 25: 31
Frame 26: 31
Frame 27: 31
Frame 28: 31
Frame 29: 31
Frame 30: 31
Frame 31: 31
Frame 32: 31
Frame 33: 31
Frame 34: 31
Frame 35: 31
Frame 36: 31
Frame 37: 31
Frame 38: 31
Frame 39: 31
Frame 40: 31
Frame 41: 31
Frame 42: 31
Frame 43: 31
Frame 44: 31
Frame 45: 31
Frame 46: 31
Frame 47: 31
Frame 48: 31
Frame 49: 30
Frame 50: 31
Frame 51: 31
Frame 52: 31
Frame 53: 31
Frame 54: 31
Frame 55: 31
Đã lưu 56 frame vào thư mục 'frames_output'.
0.5944262964290569


In [5]:
objCollision(video_path)

Chạm nhau tại: 1960.0, Frame: 49
không chạm nhau
